# Understanding Iterators and Generators

## Iterators
An iterator is basically any python type that lets you loop over a object, say using a `for` loop

What they allows you to do is :
- Have a cleaner code
- Save resources

Here is an example of a `for` loop

In [3]:
# range(5) itself is an iterator object, list is used to be able to print it
numbers = list(range(5))
print(numbers)

[0, 1, 2, 3, 4]


In [4]:
for i in numbers:
    print(i)

0
1
2
3
4


In this example, the __range(5)__ is the iterable object that provides, at each iteration, a different value that is assigned to the "i" variable.

In Python, an iterator is an object which implements the iterator protocol. 
- \__iter\__ : This returns the iterator object itself
- next() : This returns the next value. And the _StopIteration_ error once all the objects have been looped through.

When you use a loop to iterate over the iterator, you don’t need to call next and you also don’t have to worry about the StopIteration exception being raised.

Using these functions, we can create our own iterator

In [5]:
it = iter(numbers)
print(it)

In [6]:
print(next(it))
print(next(it))

0
1


In [7]:
print(list(it))

[2, 3, 4]


In [8]:
next(it) #StopIteration Error

StopIteration: 

Iterator objects can also be strings

In [9]:
string = 'python'
it = iter(string)
print(next(it))
print(next(it))
print(list(it))

p
y
['t', 'h', 'o', 'n']


In [10]:
for i in string:
    print(i, end=" ")

p y t h o n 

You can loop through multiple lists using zip

In [11]:
for i, x in zip(range(3), 'abc'):  # I think zip is a type of generator function ?
    print(i, x)

0 a
1 b
2 c


You can use `break` to exit `for` loop

In [12]:
for i in range(4000):
    print(i)
    if i >= 2:
        break

0
1
2


### Building an iterator from scratch

In [13]:
class simple_iterator:
    def __init__(self, maxnumber):
        self.maxnumber = maxnumber
        self.current = 0

    def __iter__(self):
        return self

    def __next__(self):
        next_value = self.current
        if next_value >= self.maxnumber:
            raise StopIteration
        self.current += 1
        return next_value

In [14]:
for i in build_a_simple_iterator(3):
    print(i)

NameError: name 'build_a_simple_iterator' is not defined

## Find frame rate of your webcam using iterators : Excercise 1

In [15]:
import cv2  # Opencv library to read images from webcam
import datetime

class open_webcam_images:
    def __init__(self, maxframes):
        # Captures videos from the webcam
        self.video_capture = cv2.VideoCapture(0)
        self.elapsedtime = 0
        self.maxframes = 10
        self.currentframe = 0
        self.starttime = datetime.datetime.now()

    def __iter__(self):
        return self

    def __next__(self):
        if self.currentframe >= self.maxframes:
            raise StopIteration

        _, frame = self.video_capture.read()
        self.elapsedtime = (datetime.datetime.now() -
                            self.starttime).total_seconds()
        self.currentframe += 1

        return self.elapsedtime, self.currentframe

    def __exit__(self):
        self.video_capture.release()

In [16]:
framestocapture = 10
for i in open_webcam_images(framestocapture):
    elapsedtime = i[0]
    print('Frame number {} : Elapsedtime {}'.format(i[1], i[0]))
    
# Find frame rate using elapsed time
framerate = framestocapture/i[0]
f'Frame rate of your webcam is {framerate:0.3f} frames/second'

Frame number 1 : Elapsedtime 0.045989
Frame number 2 : Elapsedtime 0.112813
Frame number 3 : Elapsedtime 0.177216
Frame number 4 : Elapsedtime 0.242821
Frame number 5 : Elapsedtime 0.308351
Frame number 6 : Elapsedtime 0.420149
Frame number 7 : Elapsedtime 0.473905
Frame number 8 : Elapsedtime 0.544556
Frame number 9 : Elapsedtime 0.608873
Frame number 10 : Elapsedtime 0.668832


'Frame rate of your webcam is 14.951 frames/second'

## Generators
Generators are just a simple form of iterators. A function that **_yields_** values.  They are a nice and compact way of building an iterator without actually building an iterator. 

The __yield__ keyword used to define generators takes care of \__iter__\() and next() for you.

__Generators are a readable shortcut for creating iterators.__

In [17]:
def simple_generator_function():
    yield 0
    yield 1
    yield 2

In [18]:
our_generator = simple_generator_function()
print(our_generator)

<generator object simple_generator_function at 0x11d8c84c0>


In [19]:
print(next(our_generator))
print(next(our_generator))
print(next(our_generator))

0
1
2


In [20]:
for i in simple_generator_function():
    print(i)

0
1
2


### Python Generators with a loop

In [21]:
def simple_generator_function(number):
    for ii in range(number):
        yield ii

In [22]:
for i in simple_generator_function(3):
    print(i)

0
1
2


### That frame rate example with generators - 

In [2]:
import cv2
import datetime
import numpy as np


def open_webcam_images(video_capture, maxframes):
    for ii in range(maxframes):
        _, frame = video_capture.read()
        yield frame


def find_framerate(maxframes):
    video_capture = cv2.VideoCapture(0)
    framenumber = 0
    starttime = datetime.datetime.now()

    for frame in open_webcam_images(video_capture, maxframes):
        elapsedtime = (datetime.datetime.now() - starttime).total_seconds()
        framenumber+=1
        print('Frame number {} : Elapsedtime {}'.format(framenumber, elapsedtime))

    video_capture.release()
    return elapsedtime


framestocapture = 10
elapsedtime = find_framerate(framestocapture)

# Find frame rate using elapsed time
framerate = framestocapture / elapsedtime
f'Frame rate of your webcam is {framerate:0.3f} frames/second'

Frame number 1 : Elapsedtime 0.046793
Frame number 2 : Elapsedtime 0.111734
Frame number 3 : Elapsedtime 0.177725
Frame number 4 : Elapsedtime 0.243822
Frame number 5 : Elapsedtime 0.310854
Frame number 6 : Elapsedtime 0.408983
Frame number 7 : Elapsedtime 0.476029
Frame number 8 : Elapsedtime 0.542221
Frame number 9 : Elapsedtime 0.608098
Frame number 10 : Elapsedtime 0.672981


'Frame rate of your webcam is 14.859 frames/second'